In [1]:
import pyspark
from pyspark.sql import *
from pyspark import SparkContext, SparkConf

In [3]:
sc = SparkSession.builder.getOrCreate().sparkContext

In [4]:
rdd = sc.parallelize([ 
  ("A", ["B", "C", "D"]),
  ("B", ["A", "C", "D", "E"]),
  ("C", ["A", "B", "D", "E"]),
  ("D", ["A", "B", "C", "E"]),
  ("E", ["B", "C", "D"])
])

In [5]:
def friends_map(x):
  seq = []
  for i in x[1] :
    if i < x[0]:
      seq.append( ((i, x[0]) , x[1]) )
    else:
      seq.append( ((x[0], i) , x[1]) )
  return seq


def friends_reduce(x, y):
    return [a for a in x if a in y]


In [6]:
similar_friends = rdd.flatMap(friends_map) \
             .reduceByKey(friends_reduce) \
             .sortByKey()
             
similar_friends.collect()

[(('A', 'B'), ['C', 'D']),
 (('A', 'C'), ['B', 'D']),
 (('A', 'D'), ['B', 'C']),
 (('B', 'C'), ['A', 'D', 'E']),
 (('B', 'D'), ['A', 'C', 'E']),
 (('B', 'E'), ['C', 'D']),
 (('C', 'D'), ['A', 'B', 'E']),
 (('C', 'E'), ['B', 'D']),
 (('D', 'E'), ['B', 'C'])]